In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict, defaultdict
import json  

In [2]:
tickers = sorted([ticker[9:-4] for ticker in glob.glob("data/raw/*.zip")])
daily_returns = pd.DataFrame(index=pd.date_range(start="1999-01-05", end="2021-03-01", 
                                                 freq=pd.tseries.offsets.BDay(), name="Date"))  # business dates
for ticker in tickers:
    df = pd.read_pickle("data/raw/" + ticker + ".zip")  # needs pickle5 compression (python 3.8)
#     print(ticker, df)
    daily_returns[ticker] = df["Log Return"].fillna(method='ffill')

daily_returns

,AA,AAIC,AAME,AAON,AAPL,AB,ABC,ABCB,ABEO,ABEV,...,Y,YPF,YUM,YVR,ZBRA,ZEUS,ZION,ZIXI,ZNH,ZTR
Date,,,,,,,,,,,,,,,,,,,,,
1999-01-05,0.022606,-0.066375,0.084558,0.012579,0.048790,-0.002430,-0.014945,0.000000,0.000000,0.000000,...,-0.001340,0.000000,0.019293,0.051293,0.016878,0.071459,-0.003114,0.006390,0.024693,0.007017
1999-01-06,0.039139,0.000000,-0.055570,0.000000,-0.036742,0.009685,0.005300,0.019609,0.000000,0.000000,...,-0.024424,0.094139,0.015171,0.000000,0.002090,0.139261,0.001039,-0.019293,0.125880,0.007863
1999-01-07,-0.012877,0.038466,0.000000,-0.012579,0.074963,-0.017012,0.042266,-0.009757,-0.054067,0.000000,...,0.017700,-0.001955,-0.005032,0.182322,0.022705,-0.030459,-0.007296,0.087011,-0.101783,-0.014084
1999-01-08,0.097711,-0.038466,0.028171,0.000000,0.000000,0.000000,-0.034368,0.014599,0.105361,0.014600,...,0.013405,-0.019764,0.002518,-0.087011,0.104544,0.030459,0.002090,0.046520,-0.036368,-0.007118
1999-01-11,0.035108,-0.009853,0.000000,0.012579,0.019256,-0.012330,0.025886,0.004819,0.139762,-0.027499,...,-0.012056,-0.010030,0.026073,0.000000,-0.045120,-0.010050,-0.027515,0.033523,0.024391,0.014185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-23,0.004024,-0.012579,-0.117531,0.010862,-0.001112,-0.017258,0.003293,0.014985,-0.060064,0.021979,...,0.009791,-0.002436,-0.003434,-0.052644,0.019690,0.028095,0.015066,-0.026344,0.141055,0.001112
2021-02-24,0.088349,0.039707,0.026847,0.013574,-0.004060,0.010445,-0.012000,0.023360,0.046520,-0.010929,...,0.040534,0.084162,0.000287,0.016086,0.029031,0.020565,0.047057,-0.015520,-0.029158,0.002220
2021-02-25,-0.062188,-0.034657,0.021835,-0.016181,-0.035402,-0.002738,-0.022592,-0.043114,0.123825,-0.041127,...,-0.005629,-0.041196,0.000191,-0.054658,-0.029835,-0.033434,-0.039700,-0.035017,-0.058117,-0.011148


In [3]:
# GENERATES A HEATMAP OF CORRELATIONS
# but its not useful since we have 1846x1846 correlations
# just uncomment everything once to try it out

# vegetables = tickers
# farmers = tickers

# harvest = daily_returns.corr(method="pearson")

# fig, ax = plt.subplots()
# im = ax.imshow(harvest)

# # We want to show all ticks...
# # ax.set_xticks(np.arange(len(farmers)))
# # ax.set_yticks(np.arange(len(vegetables)))
# # ... and label them with the respective list entries
# # ax.set_xticklabels(farmers)
# # ax.set_yticklabels(vegetables)

# # Rotate the tick labels and set their alignment.
# plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
#          rotation_mode="anchor")

# # Loop over data dimensions and create text annotations.
# for i in range(len(vegetables)):
#     for j in range(len(farmers)):
#         text = ax.text(j, i, harvest[i, j],
#                        ha="center", va="center", color="w")

# ax.set_title("Harvest of local farmers (in tons/year)")
# fig.tight_layout()
# plt.show()

In [4]:
corr = daily_returns.corr(method="pearson")

In [21]:
th = 0.834 #highest threshold for 20 pairs

def add_value(pairs_dict, key, value):
    if key in pairs_dict.keys():
        pairs_dict[key] += [value]
    else:
        pairs_dict[key] = [value]

def get_top_pairs(useThreshold):
    pairs_dict = {}
    for ticker in tickers:
        top5 = corr.sort_values(by=[ticker])[ticker][-6:-1] # exclude itself
        assert top5.name == ticker

        # TOP 5 FOR EVERY TICKER
        if not useThreshold:
            pairs_dict[ticker] = top5.to_dict({})
            
        # JUST ADD THOSE ABOVE THRESHOLD
        else:
            if top5.max() > th:

                for tick in top5.index:
                    if top5[tick] > th:
                        add_value(pairs_dict, top5.name, (tick, top5[tick]))
                
    return pairs_dict
        

In [22]:
top5alltickers = get_top_pairs(False)
top20pairs = get_top_pairs(True)

In [23]:
# SAVES THE JSON TO THE DESIRED PATH
s = json.dumps(top5alltickers)
save_path = "data/info/corellations.json"
open(save_path,"w").write(s)

268887

In [24]:
# SAVES THE JSON TO THE DESIRED PATH
j = json.dumps(top20pairs)
save_path = "data/info/top20corellations.json"
open(save_path,"w").write(j)

1366

In [25]:
# num pairs non repeating
def num_pairs(dict_):
    total = 0
    for i in dict_:
        total += len(dict_[i])
    return total / 2 # if this yields .5 anywhere thats a yikes

In [28]:
print(num_pairs(top5alltickers), num_pairs(top20pairs))

4615.0 20.0
